### Full POP model works

In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

 we can possibly look for jump operator of the form:

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12} \\ a_{21} + i b_{21} & -a_{11} - i b_{11} 
   \end{pmatrix} 
$

or

$
A = \begin{pmatrix} a_{11} & a_{12} + i b_{12}  \\ a_{21} + i b_{21} & -a_{11}
   \end{pmatrix} 
$

or assuming zero temperature 

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12}   \\ 0 & -a_{11} - i b_{11}
   \end{pmatrix} 
$

In [2]:
method = "Anszatz_"

"Anszatz_"

In [3]:
pwd()

"/home/zah/POP23"

In [4]:
parentdir = pwd()
data_dir = parentdir*"/DATA/"
println(data_dir)

models_dir = parentdir*"/MODELS/"
tests_dir = parentdir*"/TESTS/"

test_files = ["State_D"*string(n) for n=1:20];

/home/zah/POP23/DATA/


### Train models loop

In [7]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "POP_testD20_"*method*date_and_time_string * ".h5"

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in γ
    
    
    Hᵃⁿᶻ = [ 25.126        0
         0             0    ]

    γ = parse(Float64, γᵢ)

    Lᵃⁿᶻ = [ 0    √γ
             0     0. + 0im  ]  
    
    H = DenseOperator(basis, Hᵃⁿᶻ)
    A = DenseOperator(basis, Lᵃⁿᶻ)
   
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 
    
    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρ, t = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρ = convert(Vector{Matrix{ComplexF64}}, ρ)

        ρᵗᵉˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρ]

        t = convert(Vector{Float64}, t)

        tᴸᴹᴱ, ρᴸᴹᴱ  = timeevolution.master(t, ρᵗᵉˢᵗ[1], H, [A])

        F = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗᵉˢᵗ, ρᴸᴹᴱ)]

        runtime = time() - start_time

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["fidelity"] = convert.(Float64, F)
            init_state_group["runtime"] = runtime
        end
                    
        print(minimum(F))
        print(" ")
    
    end
                
    println()

end

State_D1 0.9993535209818948 State_D2 0.9993568168681232 State_D3 0.9993566146266514 State_D4 0.9993583619106424 State_D5 0.9991667249320193 State_D6 0.9991667103834582 State_D7 0.9990294909737631 State_D8 0.9990308165190603 State_D9 0.9998429434048567 State_D10 0.9998759992294537 State_D11 0.9993566056750408 State_D12 0.9993583588428344 State_D13 0.9993534766343897 State_D14 0.9993567964461711 State_D15 0.9991673053216346 State_D16 0.9991672665300837 State_D17 0.9990294750145047 State_D18 0.9990308116016056 State_D19 0.9998760039846464 State_D20 0.9998429205954843 
State_D1 0.9995515206158551 State_D2 0.9995635653029225 State_D3 0.9998327239937868 State_D4 0.9998290175308283 State_D5 0.9996426859599884 State_D6 0.9996430642834534 State_D7 0.99973601269162 State_D8 0.9997359215408272 State_D9 0.9993877454203548 State_D10 0.9998956353576357 State_D11 0.9998327204167259 State_D12 0.9998290209056752 State_D13 0.9995514767082383 State_D14 0.9995630656917287 State_D15 0.9997867444528943 Stat

In [6]:
tests_data_file_name

"POP_testD20_Anszatz_2023-Jul-29_at_20-31.h5"